In [ ]:
import numpy as np
import h5py
import scipy.io
from sklearn import metrics
import pandas as pd
import os
os.environ['THEANO_FLAGS'] = "device=cuda0,force_device=True,floatX=float32,gpuarray.preallocate=0.3"
import theano
print(theano.config.device)
from keras.layers import Embedding
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, Layer, merge, Input, Concatenate, Reshape, concatenate,Lambda,multiply,Permute,Reshape,RepeatVector
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.pooling import GlobalMaxPooling1D
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import Bidirectional, TimeDistributed
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import optimizers
from keras import backend as K
from keras import regularizers
from keras.layers import GRU
 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
sequence_input = Input(shape=(1000,4))
 
# Convolutional Layer
output = Conv1D(320,kernel_size=26,padding="valid",activation="relu")(sequence_input)
output = MaxPooling1D(pool_size=13, strides=13)(output)
output = Dropout(0.2)(output)
 
#Attention Layer
attention = Dense(1)(output)
attention = Permute((2, 1))(attention)
attention = Activation('softmax')(attention)
attention = Permute((2, 1))(attention)
attention = Lambda(lambda x: K.mean(x, axis=2), name='attention',output_shape=(75,))(attention)
attention = RepeatVector(320)(attention)
attention = Permute((2,1))(attention)
output = multiply([output, attention])
 
#GRU Layer
output = Bidirectional(GRU(320,return_sequences=True))(output)
output = Dropout(0.5)(output)
 
 
flat_output = Flatten()(output)
 
#FC Layer
FC_output = Dense(695)(flat_output)
FC_output = Activation('relu')(FC_output)
 
#Output Layer
output = Dense(690)(FC_output)
output = Activation('sigmoid')(output)
 
model = Model(inputs=sequence_input, outputs=output)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.007)
loss_fn = keras.losses.BinaryCrossentropy(from_logits=False)

In [ ]:
batch_size = 100
trainmat = h5py.File('/content/deepsea_train/train.mat')
x = trainmat['trainxdata']
y = trainmat['traindata']

optimizer = keras.optimizers.Adam(learning_rate=0.0079)
loss_fn = keras.losses.BinaryCrossentropy(from_logits=False)

In [ ]:
epochs = 60
loss_value = 0.0
for epoch in range(epochs):
  print("\nStart of epoch %d" % (epoch,))
 
  for i in range(44000):
    x_batch_train = np.transpose(np.array(x[:,:,i * 100 : (i+1) * 100]),axes=(2,0,1))
    y_batch_train = np.transpose(np.array(y[:,i * 100 : (i+1) * 100]),axes=(1,0))
    y_batch_train = y_batch_train[:,125:815]
 
    with tf.GradientTape() as tape:
 
      logits = model(x_batch_train, training=True)
 
      loss_value = loss_fn(y_batch_train, logits)
 
    grads = tape.gradient(loss_value, model.trainable_weights)
 
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
 
    if(i % 43999 == 0):
      print("Training loss at epoch %d: %.4f" % (epoch, float(loss_value)))
    if(i == 43999):
      model.save('/content/DeepTF.h5')  